# Creating pipelines

Here we explain how to create custom pipelines in nerfactory. Pipelines are composed of two components, namely a Dataloader and a Model. We'll show how to make an incremental dataloader with scene conditioning.

In [ ]:
# HIDDEN
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Dict, Optional
import torch

from nerfactory.dataloaders.base import Dataloader
from nerfactory.models.base import Model
from nerfactory.pipelines.base import Pipeline

from nerfactory.models.instant_ngp import NGPModel
from nerfactory.cameras.rays import RayBundle


class IncrementalDataloader(Dataloader):
    """A SLAM-based dataloader."""

    def populate_train_modules(self):
        """Populate the train dataloader modules."""
        # self.train_image_dataset = ImageDataset
        # self.train_image_sampler = CacheImageSampler
        # self.train_pixel_sampler = PixelSampler
        # self.train_ray_generator = RayGenerator

    def populate_eval_modules(self):
        """Populate the eval dataloader modules."""
        # self.eval_image_dataset = ImageDataset
        # self.eval_dataloader = FixedIndicesEvalDataloader

    def next_train(self, step: int):
        """Get the next batch of training data by stringing together the train modules."""
        pass

    def next_eval(self, step: int):
        """Get the next batch of eval data by stringing together the eval modules."""
        pass


class SceneConditionNGPModel(Model):
    """An instant ngp model modified slightly to output semantics."""

    def populate_modules(self):
        self.ngp_model = NGPModel()

    def get_outputs(self, ray_bundle: RayBundle, batch: Optional[Dict[str, torch.Tensor]] = None) -> Dict[str, torch.Tensor]:
        # TODO(ethan): pass in batch from the forward function
        # TODO(ethan): rename batch to something else
        outputs = self.ngp_model.forward(ray_bundle)
        outputs["semantics"] = torch.rand_like(outputs["rgb"])
        return outputs

    def get_loss_dict(self):
        return {}


class CustomPipeline(Pipeline):
    """The Instant NGP pipeline."""

    # NOTE(ethan): what does this do in Python?
    dataloader: IncrementalDataloader
    model: SceneConditionNGPModel
    # viewer: Viewer # TODO(ethan): do we want the viewer logic inside the pipeline? not sure?

    def next_train(self, step: int):
        ray_bundle, batch = self.dataloader.next_train(step=step)
        # TODO: maybe run a CNN on the data before passing into model
        model_outputs, loss_dict, metrics_dict = self.model(ray_bundle, batch)
        return model_outputs, loss_dict, metrics_dict

    def next_eval(self):
        raise NotImplementedError



In [ ]:
dataloader = Dataloader(use_train=True, use_eval=True)
model = SemanticNGPModel()
pipeline = CustomPipeline(dataloader=dataloader, model=model)

# Creating pipelines from a config

Now we show how to create a pipeline from a config, which has the following form:

```python
@dataclass
class PipelineConfig:
    """Configuration for pipeline instantiation"""

    _target_: str = MISSING
    dataloader_config: DataloaderConfig = MISSING
    graph_config: GraphConfig = MISSING
```

See [nerfactory/utils/config.py](nerfactory/utils/config.py) for more details. In this example, we will simply load from an existing configuration from [configs/graph_instant_ngp.yaml](configs/graph_instant_ngp.yaml).

In [ ]:
import pprint
import hydra

hydra.core.global_hydra.GlobalHydra.instance().clear()
from hydra import compose, initialize

initialize(version_base="1.2", config_path="../configs/")
config_name = "graph_instant_ngp.yaml"
config = compose(config_name)
pipeline_config = config.pipeline
pprint.pprint(pipeline_config)
print("----------------------------------------------------")

from nerfactory.pipelines.base import setup_pipeline

pipeline = setup_pipeline(pipeline_config, device="cuda")

In [ ]:
pipeline.get_train_loss_dict()